In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as xccrs
import cartopy.feature as cfeature
%matplotlib inline

import momlevel

## Data I/O using Dask: probably necessary

## read in monthly SSH files for an HR cycle

In [4]:
import os, re

def get_filenames_in_range(path_to_directory, lower_bound, upper_bound):
    files = []
    # Iterate through files in current directory
    for f in os.listdir(path_to_directory):
        if f.endswith(".nc"):            # print(f)        
            # Perform a regex match for numbers
            match = re.findall(r'\d+', f)
            # If numbers are found, check whether they are within range
            if match:
                number = int(match[-2])
                if number >= lower_bound and number <= upper_bound:
                    files.append(os.path.join(diri, f))
    return files

def preprocessor(d0):
    d0 = d0[['SSH']]
    # d0 = cal.time_set_midmonth(d0,'time')
    # d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

#HR cycle 3 #1993-2018
ncycle=3
startyear=1993-1958+1
nyears=2018-1993
startnumber=startyear+(ncycle-1)*61
diri = '/glade/campaign/collections/cmip/CMIP6/iHESP/FOSI/HR/g.e20.G.TL319_t13.control.001_contd/ocn/hist'
fin=get_filenames_in_range(diri,startnumber,startnumber+nyears)
len(fin)/12

26.0

In [ ]:
ds = xr.open_mfdataset(fin,combine='by_coords',coords="minimal",compat="override",
                       preprocess=preprocessor,
                       parallel=True,)

In [ ]:
ds

In [ ]:
fosi=ds.where(ds.SSH < 1e5)
geolon=fosi.TLONG.drop_vars(['TLONG','TLAT','ULONG','ULAT'])
geolat=fosi.TLAT.drop_vars(['TLONG','TLAT','ULONG','ULAT'])
omask=xr.where(np.isnan(fosi.SSH[0,:,:]), 0, 1)
omask=omask.drop_vars(['TLONG','TLAT','ULONG','ULAT'])

### process FOSI at TG locations

In [ ]:
tgcsvin="./real_tgs_in_small.csv"
# tgcsvin="./pseudo_tgs_in.csv"
tgs_in=pd.read_csv(tgcsvin)
tgs_in

In [ ]:
fosi_tg_ts=momlevel.extract_tidegauge(fosi.SSH.drop_vars(['TLONG','TLAT','ULONG','ULAT']), geolon, geolat, mask=omask, csv=tgcsvin)

In [ ]:
fosi_in=tgs_in.copy()
fosi_in.index=fosi_in['name']
fosi_in=fosi_in.T

for colname in fosi_in:
    fosi_in[colname].lat=fosi_tg_ts[colname].model_coords[0]
    fosi_in[colname].lon=fosi_tg_ts[colname].model_coords[1]
    # print(colname)
fosi_in.to_pickle("./real_hr3_tg_locs_small.pkl")
# fosi_in.to_pickle("./pseudo_hr3_tg_locs.pkl")

In [ ]:
%%time

tg_ibcorr=pd.read_pickle("tg_ts_93_18_small.pkl")

fosi_ibcorr=fosi_tg_ts.to_dataframe()
fosi_ibcorr=fosi_ibcorr.T
fosi_ibcorr.index=fosi_ibcorr.index.astype('str')
fosi_ibcorr=fosi_ibcorr.T
fosi_ibcorr.columns
fosi_ibcorr.index=tg_ibcorr.index

fosi_ibcorr.to_pickle("./real_hr3_ts_small.pkl")
# fosi_ibcorr.to_pickle("./pseudo_hr3_ts.pkl")